In [ ]:
import json
import os
import time
import math
import tkinter as tk
from tkinter import messagebox
from dataclasses import dataclass, asdict

SAVE_FILE = "plant_state.json"
TICK_MS = 1000             
AUTOSAVE_EVERY = 5    


DECAY = {
    "moisture": 0.18,
    "sunlight": 0.14,
    "nutrients": 0.06,
}


BOOSTS = {
    "water": 32,
    "sun": 24,
    "fert": 22,
}


FERT_COOLDOWN = 6 * 60 * 60


@dataclass
class PlantState:
    created_at: float
    last_tick: float
    age_seconds: float
    height: float            
    moisture: float          
    sunlight: float          
    nutrients: float         
    health: float           
    last_fertilized: float

    def clamp(self):
        self.moisture = max(0, min(100, self.moisture))
        self.sunlight = max(0, min(100, self.sunlight))
        self.nutrients = max(0, min(100, self.nutrients))
        self.health = max(0, min(100, self.health))
        self.height = max(0, min(100, self.height))


class VirtualPlantApp:
    def __init__(self, root):
        self.root = root
        self.root.title("🌱 Virtual Plant")
        self.root.geometry("760x520")
        self.root.minsize(700, 500)
        self.root.configure(bg="#0b1220")  

        self.last_autosave = time.time()

        self.state = self.load_state()

        self.build_ui()
        self.update_ui(force=True)
        self.root.protocol("WM_DELETE_WINDOW", self.on_close)
        self.tick()  

  
    def load_state(self) -> PlantState:
        if os.path.exists(SAVE_FILE):
            try:
                with open(SAVE_FILE, "r", encoding="utf-8") as f:
                    data = json.load(f)
                st = PlantState(**data)
                st.clamp()
                return st
            except Exception:
                messagebox.showwarning("Load error", "Save file was corrupt. Starting a new plant.")
        now = time.time()
        return PlantState(
            created_at=now,
            last_tick=now,
            age_seconds=0.0,
            height=8.0,
            moisture=70.0,
            sunlight=65.0,
            nutrients=60.0,
            health=80.0,
            last_fertilized=0.0,
        )

    def save_state(self):
        try:
            with open(SAVE_FILE, "w", encoding="utf-8") as f:
                json.dump(asdict(self.state), f, indent=2)
        except Exception as e:
            print("Save failed:", e)


    def build_ui(self):
      
        top = tk.Frame(self.root, bg="#0b1220")
        top.pack(fill=tk.X, padx=16, pady=(12, 6))

        title = tk.Label(top, text="Virtual Plant", fg="white", bg="#0b1220",
                         font=("Segoe UI", 20, "bold"))
        title.pack(side=tk.LEFT)

        self.status_lbl = tk.Label(top, text="", fg="#cbd5e1", bg="#0b1220",
                                   font=("Segoe UI", 11))
        self.status_lbl.pack(side=tk.RIGHT)

       
        main = tk.Frame(self.root, bg="#0b1220")
        main.pack(fill=tk.BOTH, expand=True, padx=16, pady=8)

        self.canvas = tk.Canvas(main, bg="#0f172a", highlightthickness=0)
        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

       
        side = tk.Frame(main, bg="#0b1220")
        side.pack(side=tk.RIGHT, fill=tk.Y)

       
        tk.Label(side, text="Health", bg="#0b1220", fg="white", font=("Segoe UI", 12, "bold")).pack(pady=(0, 6))
        self.health_bar = tk.Canvas(side, width=24, height=180, bg="#0f172a", highlightthickness=0)
        self.health_bar.pack(pady=(0, 12))


        self.make_meter(side, "Moisture", "moisture")
        self.make_meter(side, "Sunlight", "sunlight")
        self.make_meter(side, "Nutrients", "nutrients")

      
        btns = tk.Frame(side, bg="#0b1220")
        btns.pack(pady=10)

        self.btn_water = tk.Button(btns, text="💧 Water", command=self.on_water,
                                   font=("Segoe UI", 11), relief=tk.FLAT, padx=16, pady=8, bg="#0ea5e9", fg="white", activebackground="#0284c7")
        self.btn_sun = tk.Button(btns, text="☀️ Sunbath", command=self.on_sun,
                                 font=("Segoe UI", 11), relief=tk.FLAT, padx=16, pady=8, bg="#f59e0b", fg="black", activebackground="#d97706")
        self.btn_fert = tk.Button(btns, text="🧪 Fertilize", command=self.on_fertilize,
                                  font=("Segoe UI", 11), relief=tk.FLAT, padx=16, pady=8, bg="#10b981", fg="black", activebackground="#059669")

        self.btn_water.grid(row=0, column=0, sticky="ew", padx=4, pady=4)
        self.btn_sun.grid(row=0, column=1, sticky="ew", padx=4, pady=4)
        self.btn_fert.grid(row=1, column=0, columnspan=2, sticky="ew", padx=4, pady=4)

       
        bottom = tk.Frame(self.root, bg="#0b1220")
        bottom.pack(fill=tk.X, padx=16, pady=(6, 14))

        self.stage_lbl = tk.Label(bottom, text="", bg="#0b1220", fg="#e2e8f0", font=("Segoe UI", 11, "bold"))
        self.stage_lbl.pack(side=tk.LEFT)

        self.btn_reset = tk.Button(bottom, text="Reset Plant", command=self.reset_plant,
                                   font=("Segoe UI", 10), relief=tk.FLAT, padx=12, pady=6, bg="#ef4444", fg="white", activebackground="#dc2626")
        self.btn_reset.pack(side=tk.RIGHT)

        self.root.after(200, self.redraw_canvas)

    def make_meter(self, parent, label, key):
        frame = tk.Frame(parent, bg="#0b1220")
        frame.pack(anchor="w", pady=4)
        tk.Label(frame, text=label, bg="#0b1220", fg="#cbd5e1", font=("Segoe UI", 10, "bold")).pack(anchor="w")
        c = tk.Canvas(frame, width=180, height=16, bg="#0f172a", highlightthickness=0)
        c.pack(anchor="w", pady=(2, 0))
        setattr(self, f"bar_{key}", c)

    def tick(self):
        now = time.time()
        dt = max(0.0, now - self.state.last_tick)
        self.state.last_tick = now
        self.state.age_seconds += dt

    
        self.state.moisture -= DECAY["moisture"] * dt
        self.state.sunlight -= DECAY["sunlight"] * dt
        self.state.nutrients -= DECAY["nutrients"] * dt

    
        avg_res = (self.state.moisture + self.state.sunlight + self.state.nutrients) / 3

     
        penalty = 0
        criticals = [r for r in (self.state.moisture, self.state.sunlight, self.state.nutrients) if r < 25]
        if criticals:
            penalty = (25 - min(criticals)) * 0.8

        bonus = 0
        if min(self.state.moisture, self.state.sunlight, self.state.nutrients) > 65:
            bonus = 6

        target_health = max(0, min(100, avg_res - penalty + bonus))
        
        self.state.health += (target_health - self.state.health) * 0.12

       
        if self.state.health > 70 and min(self.state.moisture, self.state.sunlight, self.state.nutrients) > 55:
            self.state.height += 0.12 * dt  # grow slowly
        elif self.state.health < 35:
            self.state.height -= 0.05 * dt  # wither

        self.state.clamp()

       
        can_fert = (time.time() - self.state.last_fertilized) >= FERT_COOLDOWN
        self.btn_fert.configure(state=(tk.NORMAL if can_fert else tk.DISABLED))

       
        if now - self.last_autosave >= AUTOSAVE_EVERY:
            self.save_state()
            self.last_autosave = now

        
        self.update_ui()
        self.root.after(TICK_MS, self.tick)

    def on_water(self):
        self.state.moisture += BOOSTS["water"]
        self.state.moisture = min(100, self.state.moisture)
        self.flash_status("Watered 💧")

    def on_sun(self):
        self.state.sunlight += BOOSTS["sun"]
        self.state.sunlight = min(100, self.state.sunlight)
        self.flash_status("Sunbath ☀️")

    def on_fertilize(self):
        if time.time() - self.state.last_fertilized < FERT_COOLDOWN:
            self.flash_status("Fertilizer on cooldown")
            return
        self.state.nutrients += BOOSTS["fert"]
        self.state.nutrients = min(100, self.state.nutrients)
        self.state.last_fertilized = time.time()
        self.flash_status("Fertilized 🧪")

    def reset_plant(self):
        if messagebox.askyesno("Reset", "Start a brand new plant? This will overwrite the save file."):
            now = time.time()
            self.state = PlantState(
                created_at=now,
                last_tick=now,
                age_seconds=0.0,
                height=8.0,
                moisture=70.0,
                sunlight=65.0,
                nutrients=60.0,
                health=80.0,
                last_fertilized=0.0,
            )
            self.save_state()
            self.update_ui(force=True)
            self.flash_status("Plant reset 🌱")

   
    def update_ui(self, force=False):
        
        self.draw_horizontal_bar(self.bar_moisture, self.state.moisture)
        self.draw_horizontal_bar(self.bar_sunlight, self.state.sunlight)
        self.draw_horizontal_bar(self.bar_nutrients, self.state.nutrients)
        self.draw_health_bar(self.health_bar, self.state.health)

        
        stage = self.get_stage()
        self.stage_lbl.config(text=f"Stage: {stage}  •  Height: {self.state.height:.1f}  •  Age: {self.format_age(self.state.age_seconds)}")

    def get_stage(self):
        h = self.state.height
        if h < 15:
            return "Seedling"
        elif h < 35:
            return "Sprout"
        elif h < 65:
            return "Young Plant"
        elif h < 90:
            return "Mature"
        else:
            return "Blooming"

    def format_age(self, secs):
        d = int(secs // 86400)
        h = int((secs % 86400) // 3600)
        m = int((secs % 3600) // 60)
        if d > 0:
            return f"{d}d {h}h"
        elif h > 0:
            return f"{h}h {m}m"
        else:
            return f"{m}m"

    def redraw_canvas(self):
        self.canvas.delete("all")
        w = self.canvas.winfo_width()
        h = self.canvas.winfo_height()

        
        self.canvas.create_rectangle(0, h-80, w, h, fill="#1f2937", outline="")

    
        pot_w = 180
        pot_h = 80
        pot_x = w/2 - pot_w/2
        pot_y = h-80 - pot_h
        self.canvas.create_rectangle(pot_x, pot_y, pot_x+pot_w, pot_y+pot_h, fill="#7c3aed", outline="")
        self.canvas.create_rectangle(pot_x, pot_y, pot_x+pot_w, pot_y+18, fill="#a78bfa", outline="")

        
        health = self.state.health
        if health >= 70:
            stem_color = "#22c55e"  
        elif health >= 40:
            stem_color = "#fbbf24"  
        else:
            stem_color = "#ef4444"  

       
        stem_height_px = 40 + (self.state.height * 3)  
        base_x = w/2
        base_y = pot_y
        top_y = max(40, base_y - stem_height_px)

       
        t = time.time()
        sway = math.sin(t * 0.8) * 16 * (0.5 + self.state.health/200)

        
        self.canvas.create_line(base_x, base_y, base_x + sway/2, (base_y+top_y)//2,
                                base_x + sway, top_y, width=6, fill=stem_color, smooth=True)

    
        for i, frac in enumerate([0.25, 0.5, 0.75], start=1):
            ly = base_y - (stem_height_px * frac)
            lx = base_x + sway * (frac - 0.5)
            leaf_size = 22 + i*2
            self.draw_leaf(lx, ly, leaf_size, stem_color)

       
        if self.state.height > 85 and self.state.health > 65:
            self.draw_flower(base_x + sway + 6, top_y - 8)

        
        self.root.after(120, self.redraw_canvas)

    def draw_leaf(self, cx, cy, size, color):
        
        self.canvas.create_oval(cx - size*1.6, cy - size*0.6, cx - size*0.2, cy + size*0.4, fill=color, outline="")
        
        self.canvas.create_oval(cx + size*0.2, cy - size*0.6, cx + size*1.6, cy + size*0.4, fill=color, outline="")

    def draw_flower(self, x, y):
        petals = 6
        r_outer = 12
        r_inner = 5
        for i in range(petals):
            ang = (2*math.pi/petals) * i
            px = x + math.cos(ang)*r_outer
            py = y + math.sin(ang)*r_outer
            self.canvas.create_oval(px-8, py-8, px+8, py+8, fill="#f472b6", outline="")
        self.canvas.create_oval(x-r_inner, y-r_inner, x+r_inner, y+r_inner, fill="#fde047", outline="")

    def draw_horizontal_bar(self, canvas, value):
        canvas.delete("all")
        w = int(canvas["width"]) or 180
        h = int(canvas["height"]) or 16
        canvas.create_rectangle(0, 0, w, h, fill="#0f172a", outline="")
        fill_w = int(w * max(0, min(100, value)) / 100)
       
        color = self.value_color(value)
        canvas.create_rectangle(0, 0, fill_w, h, fill=color, outline="")
        canvas.create_text(w-6, h//2, text=f"{value:5.1f}%", anchor="e", fill="#e5e7eb", font=("Segoe UI", 9, "bold"))

    def draw_health_bar(self, canvas, value):
        canvas.delete("all")
        w = int(canvas["width"]) or 24
        h = int(canvas["height"]) or 180
        canvas.create_rectangle(0, 0, w, h, fill="#0f172a", outline="")
        fill_h = int(h * max(0, min(100, value)) / 100)
        color = self.value_color(value)
        canvas.create_rectangle(0, h - fill_h, w, h, fill=color, outline="")
        # tick lines
        for i in range(0, 101, 20):
            y = h - int(h * i / 100)
            canvas.create_line(0, y, w, y, fill="#1f2937")

    def value_color(self, value):
        if value >= 70:
            return "#22c55e"  
        elif value >= 40:
            return "#fbbf24"  
        else:
            return "#ef4444" 

    def flash_status(self, text):
        self.status_lbl.config(text=text)
        self.root.after(1400, lambda: self.status_lbl.config(text=""))

    def on_close(self):
        self.save_state()
        self.root.destroy()


if __name__ == "__main__":
    root = tk.Tk()
    app = VirtualPlantApp(root)
    root.mainloop()
